TODO: turn this into a unit test

In [1]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

from mlp_framework_vanilla import *

In [2]:
test_net = Net(2, [2, 1], [sigmoid]*2, log_loss)

training = [
    {'x': [0,0], 'y': [0]},
    {'x': [0,1], 'y': [1]},
    {'x': [1,1], 'y': [0]},
]
x = [2, 1]
y = [1]

In [3]:
test_net.train(training, epochs=1_000, epoch_progress_every=500)
activations = test_net.forward(x)
gradients = test_net.backward(x, y, activations)

epoch  500 complete, 1 batch, training_loss=0.52770
epoch 1000 complete, 1 batch, training_loss=0.20499


In [4]:
gradients['weight']

[[[0.38912063495300986, 0.19456031747650493],
  [0.15677245453900926, 0.07838622726950463]],
 [[-0.8968413039772322, -0.9573849401356327]]]

In [5]:
def _sigmoid(v): # devectorize
    return sigmoid([v])[0]

# use finite difference approximation to check our gradient algorithms:
e = 0.0001

# Simulate a normal forward pass:
# Layer 1 Node 1:
l1n1a = _sigmoid(x[0] * test_net.layers[0].nodes[0].weights[0] + \
                 x[1] * test_net.layers[0].nodes[0].weights[1] + \
                        test_net.layers[0].nodes[0].bias + 0)
# Layer 1 Node 2:
l1n2a = _sigmoid(x[0] * test_net.layers[0].nodes[1].weights[0] + \
                 x[1] * test_net.layers[0].nodes[1].weights[1] + \
                        test_net.layers[0].nodes[1].bias)
# Layer 2 Node 1 (output node):
y_hat = _sigmoid(l1n1a * test_net.layers[1].nodes[0].weights[0] + \
                 l1n2a * test_net.layers[1].nodes[0].weights[1] + \
                         test_net.layers[1].nodes[0].bias)
loss = log_loss(y, [y_hat])

# Introduce a difference to some one term (move around the 'e'):
l1n1a = _sigmoid(x[0] * (test_net.layers[0].nodes[0].weights[0] + e) + \
                 x[1] * (test_net.layers[0].nodes[0].weights[1] + 0) + \
                        (test_net.layers[0].nodes[0].bias + 0))
l1n2a  = _sigmoid(x[0] * (test_net.layers[0].nodes[1].weights[0] + 0) + \
                  x[1] * (test_net.layers[0].nodes[1].weights[1] + 0) + \
                         (test_net.layers[0].nodes[1].bias + 0))
y_hat = _sigmoid(l1n1a * (test_net.layers[1].nodes[0].weights[0] + 0) + \
                 l1n2a * (test_net.layers[1].nodes[0].weights[1] + 0) + \
                         (test_net.layers[1].nodes[0].bias + 0))
loss_e = log_loss(y, [y_hat])

print (f'dL/d[e] = {(loss_e - loss)/e}')

dL/d[e] = 0.3890885374291386
